In [1]:
from pathlib import Path

import typer
import pandas as pd
import verifiers as vf
from dotenv import load_dotenv
from openai import AsyncOpenAI


assert load_dotenv(), "Failed to load .env file"

In [2]:
import mlflow

# Tell MLflow about the server URI.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# Enable autologging with all features
mlflow.openai.autolog()
# Create a unique name for your experiment.
mlflow.set_experiment("rlvr-eval")

<Experiment: artifact_location='mlflow-artifacts:/2', creation_time=1756636926792, experiment_id='2', last_update_time=1756636926792, lifecycle_stage='active', name='rlvr-eval', tags={}>

In [3]:
openai_client = AsyncOpenAI()

models_response = await openai_client.models.list()
available_models = [item.id for item in models_response.data if item.object == "model"]
print("Available models:", available_models)

model = available_models[0]
print("Using model:", model)


Available models: ['willcb/Qwen3-8B']
Using model: willcb/Qwen3-8B


In [4]:
datasets_str = "bdsaglam/musique-mini,answerable,validation"
noise_rate = 1.0
retriever = "hybrid"
temperature = 0.5
max_new_tokens = 1024

print("🔮 Starting MuSiQue evaluation")
print("=" * 50)
print(f"📝 Model: {model}")
print(f"📊 Dataset: {datasets_str} (noise rate: {noise_rate})")
print(f"🔍 Retriever: {retriever}")
print(f"🌡️ Temperature: {temperature}")
print(f"🎯 Max tokens: {max_new_tokens}")
print("=" * 50)

# Load MuSiQue environment
print("🌍 Loading MuSiQue environment...")

vf_env = vf.load_environment(
    env_id="vf-musique",
    datasets_str=datasets_str,
    noise_rate=noise_rate,
    retriever_name=retriever,
)
print(f"✅ Environment loaded with {len(vf_env.dataset)} examples")

# Use OpenAI-compatible API client (e.g., for vLLM)
print("🤖 Using OpenAI-compatible API client...")

# Run evaluation using the environment
print("🔄 Running evaluation...")
results = vf_env.evaluate(
    openai_client,
    model,
    rollouts_per_example=1,
    sampling_args={"temperature": temperature, "max_tokens": max_new_tokens},
)


🔮 Starting MuSiQue evaluation
📝 Model: willcb/Qwen3-8B
📊 Dataset: bdsaglam/musique-mini,answerable,validation (noise rate: 1.0)
🔍 Retriever: hybrid
🌡️ Temperature: 0.5
🎯 Max tokens: 1024
🌍 Loading MuSiQue environment...


Map: 100%|##########| 300/300 [00:00<?, ? examples/s]

Filter:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

2025-09-02 22:48:25 - verifiers.envs.MuSiQueEnv - INFO - eval_dataset is not set, falling back to train dataset


✅ Environment loaded with 100 examples
🤖 Using OpenAI-compatible API client...
🔄 Running evaluation...


Evaluating 100 rollouts: 100%|██████████| 100/100 [00:00<00:00, 690.56it/s]


[Trace(trace_id=tr-27c1e94a45197290cba78dc06610404a), Trace(trace_id=tr-5d02084f200085d432a2aadeb7d18c6c), Trace(trace_id=tr-16f7a5607e4d90f5ea12ad99308881fd), Trace(trace_id=tr-3d8c7a00a4d363df6e7c12f1226ce377), Trace(trace_id=tr-3e8de33d4e8502b97a754c4c86106323), Trace(trace_id=tr-a3f06c1f9cab9cef1bcc651afe2f005d), Trace(trace_id=tr-ff5d5295877d6dd43fb7f31b620a761e), Trace(trace_id=tr-3a57ff90b1c162b0acb04aa20bff95cf), Trace(trace_id=tr-17a3b5c649191bbfa43c87c0588dd43c), Trace(trace_id=tr-a0f96012a60d82c02348e99ef7532150)]

In [5]:
results.metrics.keys()

dict_keys(['exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'])

In [6]:
from verifiers.types import GenerateOutputs

for field in GenerateOutputs.__fields__.keys():
    print(field)

    _type = type(getattr(results, field))
    if _type == list:
        print(getattr(results, field)[0])
    else:
        print(type(getattr(results, field)))

prompt
[{'role': 'system', 'content': "Answer the question based on the information provided by tools.\n\nFor each step:\n1. Think through your reasoning inside <think> tags\n2. Use tools to retrieve documents\n3. Continue until you find the answer through multi-hop reasoning. The question is answerable from the docs. \n4. In the **last** step:\n    - Reflect on your previous steps inside <think> tags\n    - Cite the documents you base your answer on inside <cite> tags by their IDs, e.g. `<cite>1, 2, 3</cite>`\n    - Give your final answer inside <answer> tags\nAn example for your final message:\n```\n<think>\n[your thinking and explanation here]\n</think> \n<cite>\n[IDs of the documents that back your answer]\n</cite>\n<answer>\n[your final answer in **a few words**. no explanation here.]\n</answer>\n```\n\n- Do not make up tools or arguments that aren't listed.\n- Make one tool call per step.\n- Questions require multi-hop reasoning across multiple documents.\n- Continue searching un

/tmp/ipykernel_2596771/3696243233.py:3: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for field in GenerateOutputs.__fields__.keys():


In [7]:
ds = vf_env.make_dataset(results)
ds

Dataset({
    features: ['prompt', 'completion', 'answer', 'task', 'reward', 'info', 'exact_match_reward', 'f1_reward', 'retrieval_recall_reward', 'retrieval_precision_reward', 'citation_reward', 'format_reward', 'combined_reward'],
    num_rows: 100
})

In [8]:
ds.to_json("../outputs/musique-eval-results.jsonl", orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

2944652

In [9]:
df = ds.to_pandas()
df

,prompt,completion,answer,task,reward,info,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
0,[{'content': 'Answer the question based on the...,"[{'content': '<think> </think> ', 'role': 'as...",LeMat revolver,default,0.175000,"{'answers': ['LeMat Revolver', 'LeMat revolver...",0.0,0.0,0.5,0.333333,0.0,0.666667,0.175000
1,[{'content': 'Answer the question based on the...,"[{'content': '<think> Okay, let's try to figur...",Vitor Constâncio,default,0.008333,"{'answers': ['vitor constâncio', 'Vitor Constâ...",0.0,0.0,0.0,0.000000,0.0,0.333333,0.008333
2,[{'content': 'Answer the question based on the...,"[{'content': '<think> Okay, let's tackle this ...",45,default,0.858333,"{'answers': ['45'], 'docs': [{'body': 'Weber C...",1.0,1.0,1.0,0.333333,0.5,1.000000,0.858333
3,[{'content': 'Answer the question based on the...,"[{'content': '<think> Okay, so the user is ask...",three different relationships he had in the past,default,0.008333,{'answers': ['three different relationships he...,0.0,0.0,0.0,0.000000,0.0,0.333333,0.008333
4,[{'content': 'Answer the question based on the...,"[{'content': '<think> Okay, so the question is...",Michael Bublé,default,0.175000,"{'answers': ['michael bublé', 'Michael Bublé']...",0.0,0.0,0.5,0.500000,0.0,0.000000,0.175000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[{'content': 'Answer the question based on the...,"[{'content': '<think> Okay, let's try to figur...",two alliances,default,0.008333,"{'answers': ['two alliances'], 'docs': [{'body...",0.0,0.0,0.0,0.000000,0.0,0.333333,0.008333
96,[{'content': 'Answer the question based on the...,"[{'content': '<think> </think> ', 'role': 'as...",John Kukuzelis,default,0.750000,"{'answers': ['John Kukuzelis', 'john kukuzelis...",1.0,1.0,0.5,0.500000,0.5,1.000000,0.750000
97,[{'content': 'Answer the question based on the...,"[{'content': '<think> </think> ', 'role': 'as...",Vito Corleone,default,0.275000,"{'answers': ['vito andolini', 'Vito Andolini',...",0.0,0.0,0.5,0.500000,0.5,1.000000,0.275000
98,[{'content': 'Answer the question based on the...,"[{'content': '<think> </think> ', 'role': 'as...",The Île de la Cité,default,0.183333,"{'answers': ['île de la cité', 'The Île de la ...",0.0,0.0,0.5,0.500000,0.0,0.333333,0.183333


In [10]:
df.describe()

,reward,exact_match_reward,f1_reward,retrieval_recall_reward,retrieval_precision_reward,citation_reward,format_reward,combined_reward
count,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000
mean,0.360469,0.240000,0.296381,0.57500,0.471167,0.190000,0.576667,0.360469
std,0.332102,0.429235,0.433115,0.39807,0.331856,0.299663,0.344444,0.332102
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.140000,0.000000,0.000000,0.37500,0.150000,0.000000,0.333333,0.140000
50%,0.191667,0.000000,0.000000,0.50000,0.500000,0.000000,0.500000,0.191667
75%,0.600000,0.000000,0.700000,1.00000,0.666667,0.500000,1.000000,0.600000
max,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [11]:
row = df.iloc[0]